# Clean : trip statistics

# Purpose
There are some outliers in the trip statistics that will need to be removed.

# Setup

In [ ]:
# %load imports.py
# %load ../imports.py
%matplotlib inline
%load_ext autoreload
%autoreload 2

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.pyplot as plt
import seaborn as sns
width=20
height=3
plt.rcParams["figure.figsize"] = (width,height)
sns.set(rc={'figure.figsize':(width,height)})

#import seaborn as sns
import os
from collections import OrderedDict

from IPython.display import display

pd.options.display.max_rows = 999
pd.options.display.max_columns = 999
pd.set_option("display.max_columns", None)

import folium
import plotly.express as px
import plotly.graph_objects as go

import sys
import os

from sklearn.metrics import r2_score

import scipy.integrate
import seaborn as sns

import pyarrow as pa
import pyarrow.parquet as pq

import dask.dataframe
import statsmodels.api as sm

from d2e2f.visualization import visualize
import scipy.integrate

from d2e2f.pipelines.trip_statistics import clean_statistics

In [ ]:
%reload_kedro
df_stat = catalog.load('uraniborg.trip_statistics_clean')
#df_stat = catalog.load('uraniborg.trip_statistics')
df_stat['start_time'] = pd.to_datetime(df_stat['start_time'], utc=True)
df_stat['end_time'] = pd.to_datetime(df_stat['end_time'], utc=True)
df_stat.describe()

In [ ]:
df_stat.head()

In [ ]:
df_stat.shape

In [ ]:
df_stat.info()

In [ ]:
sns.displot(df_stat, x='distance', aspect=3);

In [ ]:
sns.displot(df_stat, x='trip_time', aspect=3);

In [ ]:
df_stat["consumption"] = df_stat["Consumption ME1 (L)"] + df_stat["Consumption ME2 (L)"]

In [ ]:
sns.displot(df_stat, x='consumption', aspect=3);

In [ ]:
df_clean = clean_statistics.clean(df_stat=df_stat, min_distance=4000, min_time=700)

In [ ]:
df_compare = df_stat.copy()
df_compare['item'] = 'raw'

df_clean['item'] = 'clean'
df_compare = df_compare.append(df_clean, ignore_index=True)

In [ ]:
df_compare.describe()

In [ ]:
sns.displot(df_compare, x='E', hue='item', binwidth=5*10**4, aspect=3)

In [ ]:
sns.displot(df_compare, x='trip_time', hue='item', binwidth=25, aspect=3)

In [ ]:
facegrid = sns.relplot(data=df_stat, x='start_time',y='consumption', height=3, aspect=3);
facegrid.set_ylabels('(Fuel consumption per trip) [l]')

In [ ]:
df_ = df_stat.set_index('start_time')
df_.sort_index(inplace=True)

df_['dt'] = np.concatenate(([0],np.diff((df_.index-df_.index[0]).total_seconds())))

mask = df_['dt'] > 60*60*24*5

mask[0]=True
df_.loc[mask,'group'] = np.arange(mask.sum())
df_['group'] = df_['group'].fillna(method='pad')

fig,ax=plt.subplots()

#df_.plot(y='dt', ax=ax)
df_.loc[mask].plot(y='dt', style='ro', ax=ax)

for group, df_group in df_.groupby(by='group'):
    
    df_group.plot(y='dt', label=group, ax=ax)


In [ ]:
window = "30D"

df_rolling = pd.DataFrame()

for group, df_group in df_.groupby(by='group'):

    df_rolling_ = df_group.rolling(window=window).mean().iloc[10:]
    df_rolling_['group'] = group
    df_rolling = df_rolling.append(df_rolling_)

In [ ]:



df_expanding = df_.expanding().mean().iloc[20:]

facegrid = sns.relplot(data=df_rolling, x='start_time', y='consumption', height=3, aspect=3);
facegrid.set_ylabels('(Fuel consumption per trip) [l]')

In [ ]:
tänd = '2021-04-01'
utbildning = '2021-05-11'

df_.loc[:tänd,'period'] = 'släckta skärmar'
df_.loc[tänd:utbildning,'period'] = 'tända skärmar'
df_.loc[utbildning:,'period'] = 'efter utbildning'

In [ ]:
fig,ax=plt.subplots()
fig.set_size_inches(20,7)

#df_before.plot(y='consumption', label='before', ax=ax, style='b.')
#df_after.plot(y='consumption', label='after', ax=ax, style='g.')

styles = ['k.','y.','b.']
for style, (period, df_period) in zip(styles,df_.groupby(by='period', sort=False)):
    df_period.plot(y='consumption', label=period, ax=ax, style=style)



df_rolling.plot(y='consumption', label=f'rolling mean ({window})', ax=ax, style='m-')
df_expanding.plot(y='consumption', label=f'expanding mean', ax=ax, style='c-')
ax.legend();

In [ ]:
fig,ax=plt.subplots()
fig.set_size_inches(20,7)

for group, df_rolling_group in df_rolling.groupby(by='group'):
    df_rolling_group.plot(y='consumption', label=f'rolling mean', ax=ax, style='-')


df_expanding.plot(y='consumption', label=f'expanding mean', ax=ax, style='-')

ax.set_ylim(0.85*df_before['consumption'].mean(),1.1*df_before['consumption'].mean())
ax.legend();

In [ ]:
interesting = ['sog','consumption','distance','aw']
df_compare = df_.groupby(by='period', sort=False).mean()[interesting]
df_compare

In [ ]:
M = np.tile(df_compare['consumption'],(len(df_compare),1))
changes = np.triu(M - M.T)
df_changes = pd.DataFrame(changes, index=df_compare.index, columns=df_compare.index)

columns = df_changes.columns[~(df_changes==0).all(axis=0)]
rows = df_changes.index[~(df_changes==0).all(axis=1)]
df_changes = df_changes.loc[rows,columns].copy()

In [ ]:
(df_changes/df_compare.loc['släckta skärmar']['consumption'])*100